In [18]:
from arftools import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import copy 
from sklearn.metrics.pairwise import rbf_kernel
import random
from sklearn.linear_model import Perceptron
from sklearn.datasets import load_digits
import tools as t

In [68]:
def mse(datax,datay,w):
    """ retourne la moyenne de l'erreur aux moindres carres """
    return np.mean((np.dot(datax, w.T) - datay)**2)

def mse_g(datax,datay,w):
    """ retourne le gradient moyen de l'erreur au moindres carres """
    return np.mean(2 * (np.dot(datax, w.T) - datay))

def hinge(datax,datay,w):
    """ retourn la moyenne de l'erreur hinge """
    return max(0,-datay * w.dot(datax))

def hinge_g(datax,datay,w):
    """ retourne le gradient moyen de l'erreur hinge """
    if np.mean(datay * np.dot(datax,w.T))<=0:
        return np.mean(-datay*datax)
    else:
        return np.zeros(w.shape)
def chunks(l, n):
    n = max(1, n)
    result=[]
    for i in range(0, len(l), n):
        result.append(l[i:i+n])
    return result

def add_bias(datax):
    new_dim = np.ones(datax.shape[0]).reshape(-1, 1)
    new_datax=np.hstack((datax, new_dim))
    return new_datax

#calculate the gaussian proj for an example x - miniX 
def gauss(x,datax, sigma):
    x_xdata=x - datax
    result=np.zeros((1,x_xdata.shape[0]))
    i=0
    for norm_s in x_xdata:
        result[0][i]=np.exp(-np.linalg.norm(norm_s, 2)**2 / (2. * sigma**2))
        i+=1
    return result

def projection(datax,kernel="poly",sigma=0,miniX=[],datay=[]):
    if kernel=="poly":
        N=datax.shape[0]
        new_datax=np.zeros((N,N))
        for i in range(len(datax)):
            x1=datax[i]
            for j in range(len(datax)):
                x2=datax[j]
                a=1+x1.dot(x2)
                new_datax[i][j]=np.power(a,datax.shape[1])
        return new_datax
class Linear(object):
    def __init__(self,loss=hinge,loss_g=hinge_g,max_iter=1000,eps=0.01, kernel="poly",sigma=0,typeUp="batch",bins=0,nbPoints=100,w_type='random'):
        """ :loss: fonction de cout
            :loss_g: gradient de la fonction de cout
            :max_iter: nombre d'iterations
            :eps: pas de gradient
            :kernel: noyau du perceptron
        """
        self.max_iter, self.eps = max_iter,eps
        self.loss, self.loss_g = loss, loss_g
        self.kernel = kernel
        self.sigma=sigma
        self.bins=bins
        self.typeUp=typeUp
        self.miniX=[]
        self.nbPoints=nbPoints
        self.w_type=w_type
        
    def fit(self,datax,datay,testx=None,testy=None):
        """ :datax: donnees de train
            :datay: label de train
            :testx: donnees de test
            :testy: label de test
        """
        # on transforme datay en vecteur colonne
        datay = datay.reshape(-1,1)
        N = len(datay)
        datax = datax.reshape(N,-1)
        D = datax.shape[1]
        if(self.w_type=='random'):
            self.w = np.random.random((1,D))
        if(self.w_type=='mean'):
            self.w = np.array([datax.mean(0)])
        self.w_init=copy.deepcopy(self.w)
        self.w_hist=[]
        data_projected=copy.deepcopy(datax)
        
        if (self.kernel=='poly'):
            self.w = np.random.random((1,D+3))
            self.w_init=copy.deepcopy(self.w)
        if(self.typeUp=='batch'):      
            for i in range(self.max_iter):
                self.dd=data_projected
                self.cc=self.loss_g(data_projected, datay, self.w)
                for j in range(len(data_projected)):
                    self.w -= self.eps * self.loss_g(data_projected[i], datay, self.w)
                    self.w_hist.append(self.w)
        elif(self.typeUp=='stochastique'):
            for i in range(self.max_iter):
                index=random.choice(np.arange(0,data_projected.shape[0],1))
                self.w -= self.eps * self.loss_g(data_projected[index], datay[index], self.w)
                self.w_hist.append(self.w)
        elif(self.typeUp=='minibatch'):  
            list_shuffle=np.arange(0,data_projected.shape[0],1)
            np.random.shuffle(list_shuffle)
            datax_s=[x for _,x in sorted(zip(list_shuffle,data_projected))]
            datay_s=[x for _,x in sorted(zip(list_shuffle,datay))]
            mini_binsX=chunks(datax_s,1000)
            mini_binsY=chunks(datay_s,1000)
            for i in range(len(mini_binsX)):
                for j in range(self.max_iter):
                    self.w -= self.eps * self.loss_g(np.array(mini_binsX[i]), np.array(mini_binsY[i]), self.w)
                    self.w_hist.append(self.w)                   

    def predict(self,datax):
        data_projected=copy.deepcopy(datax)
        if len(datax.shape)==1:
            datax = datax.reshape(1,-1)
        if(self.kernel=='poly'):
            data_projected=self.project_data(copy.deepcopy(datax),None)
        return np.sign(np.dot(data_projected, self.w.T)).reshape(-1)

    def score(self,datax,datay):
        return np.where(self.predict(datax).reshape(-1)==datay,1,0).mean()
    def project_data(self,datax,datay):
        return projection(datax,self.kernel,sigma=self.sigma,miniX=self.miniX,datay=datay)    


<center> <h1> Introduction : Module scikit-learn </h1></center>

In [19]:
datax_train, datay_train = load_digits(return_X_y=True)

In [20]:
p1= Perceptron(tol=0.01, random_state=0)
p1.fit(datax_train, datay_train)
p1.score(datax_train,datay_train)

0.9460211463550362

In [53]:
p2= Linear(hinge,hinge_g,max_iter=1000,eps=0.001,typeUp='minibatch')
p2.fit(datax_train,datay_train)
p2.score(datax_train,datay_train)

0.10127991096271564

<center> <h1> Linéaire pénalisé - régularisation de Tikhonov</h1></center>

<center> <h1> SVM etGrid Search </h1></center>

In [54]:
trainx,trainy =  gen_arti(nbex=1000,data_type=0,epsilon=1)
testx,testy =  gen_arti(nbex=1000,data_type=0,epsilon=1)

<center> <h1> Apprentissage multi-classe </h1></center>

<center> <h1> String Kernel  </h1></center>